In [1]:
### Setup environment

!mkdir plots

### Install and load packages
 
#!pip install uncertainties     # uncomment to install dependancy
#!pip install lmfit             # for docs see https://lmfit.github.io/lmfit-py/


import lmfit

from scipy.optimize import curve_fit     # tool for curve fitting
import scipy                             # includes tools for data analysis
import scipy.stats
import numpy as np                       # import the tools of NumPy 
from matplotlib import pyplot as plt     # tools for plotting
import pandas as pd

import uncertainties as un               # tool set for handling  uncertainties
from uncertainties import unumpy as unp  # a replacement for numpy 

### Set global variables

location_data = "data/"         ## Use either local folder or github folder. 
location_styles = "styles/"     ## Use github locations for Colab
#location_data = "https://raw.githubusercontent.com/blinkletter/PythonPresentation/main/data/"
#location_styles = "https://raw.githubusercontent.com/blinkletter/PythonPresentation/main/styles/"

size = [4,4]
size2 = [4,5]
size3 = [2.5,2.5]





### LMFit

######################
### Read in data set
######################

datafile = "Eryingdata1.csv"   # change this file for new data
#datafile = "Eryingdata2.csv"   # change this file for new data
#datafile = "Eryingdata3.csv"   # change this file for new data
#datafile = "Eryingdata4.csv"   # change this file for new data

data = pd.read_csv(location_data+datafile, 
                 delimiter = ",", 
                 skipinitialspace=True, 
#                 index_col="TEMP(K)", 
                 comment = "#") 
display(data)

######################
### Experimental data with error
######################

temp = data["TEMP(K)"]       # list of temperatures
k_obs = data["Kcat(s-1)"]      # list of observe rate constants (s^-1)
k_obs_err = data["Error(s-1)"]  # list of standard deviations for data

### Convert lists to numpy arrays (enables numpy math tools with these lists)
temp = np.array(temp)
k_obs_u = unp.uarray(k_obs, k_obs_err)   # make an array of ufloat values

### Calculations for Erying plot axes
x = 1/temp
y_u = unp.log(k_obs_u/temp)   # y_u is now an array of ufloats

y = unp.nominal_values(y_u) # extract arrays of nominal values and errors
y_err = unp.std_devs(y_u)   # because curve_fit can handle ufloats

######################
###  Linear function to be used by curve_fit
######################
def linear(x, m, b):
    y = m * x + b
    return y

### Use curve_fit function  

# load the function f as the model to be fit
mod = lmfit.Model(linear)       

# state the parameters (use the text strings that we used above in the 
#   function) initial values are also set here
pars = mod.make_params(m=-1000, b=-1  )     
                                            
# use the .fit method on the model object to perform the curve fit
result = mod.fit(y, pars, x=x, weights=1.0/y_err)   


print("---------- FIT REPORT ------------")
print(result.fit_report())

mkdir: plots: File exists


,TEMP(K),Kcat(s-1),Error(s-1)
0,283,3.4,0.1
1,288,5.3,0.2
2,293,7.6,0.2
3,298,11.7,0.3
4,303,15.2,0.1
5,308,21.3,0.9
6,313,27.8,0.9
7,318,39.0,3.0
8,323,52.0,4.0
9,325,61.0,2.0


---------- FIT REPORT ------------
[[Model]]
    Model(linear)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 7
    # data points      = 12
    # variables        = 2
    chi-square         = 17.4581508
    reduced chi-square = 1.74581508
    Akaike info crit   = 8.49879980
    Bayesian info crit = 9.46861310
    R-squared          = -0.57535644
[[Variables]]
    m: -5857.88411 +/- 89.2802914 (1.52%) (init = -1000)
    b:  16.3401636 +/- 0.29494937 (1.81%) (init = -1)
[[Correlations]] (unreported correlations are < 0.100)
    C(m, b) = -0.9997


In [4]:
result.best_fit

array([-4.35907354, -3.99971178, -3.65261493, -3.31716563, -2.99278726,
       -2.67894065, -2.37512109, -2.08085561, -1.79570051, -1.6840952 ,
       -1.51923917, -1.41100037])

In [5]:
dir(result)

['_Minimizer__jacobian',
 '_Minimizer__residual',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abort',
 '_calculate_covariance_matrix',
 '_calculate_uncertainties_correlations',
 '_err_max_evals',
 '_err_nonparam',
 '_int2ext_cov_x',
 '_lnprob',
 '_repr_html_',
 'aborted',
 'aic',
 'ampgo',
 'basinhopping',
 'best_fit',
 'best_values',
 'bic',
 'brute',
 'calc_covar',
 'call_kws',
 'chisqr',
 'ci_out',
 'ci_report',
 'col_deriv',
 'components',
 'conf_interval',
 'covar',
 'data',
 'dual_annealing',
 'dump',
 'dumps',
 'emcee',
 'errorbars',
 'eval',
 'eval_components',
 'eval_uncertainty',
 'fit',
 'fit_report',
 'flatchain',
 'ier',
 'init_fit',

In [14]:
result.eval(x=np.array([1,2,3]))

array([ -5841.54395115, -11699.42806592, -17557.31218069])